In [ ]:
##########################################################################################################
#              Exercício 1 (parte 1) - implementação do FBA
#     Com base no modelo estequiométrico iML1515 de E.coli substr. K-12 no formato SBML foi 
#     efetuada uma verificação da produção wild-type do composto de interesse: o acetato. 
#
# Grupo 2: 
#     Diogo Cachetas, João Monteiro, Juliana Pereira, Joana Gabriel, Sara Boaventura, Teresa Coimbra
##########################################################################################################

In [1]:
from mewpy.simulation import get_simulator
from cobra.io import read_sbml_model
from cobra.flux_analysis import flux_variability_analysis
import pandas
from time import time
import cobra.test
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

In [2]:
model = read_sbml_model('./iML1515.xml')                # criação do modelo
print(model.summary())                                  # sumário do modelo inicial sem alteração das condições ambientais

Objective
1.0 BIOMASS_Ec_iML1515_core_75p37M = 0.8769972144269693

Uptake
------
Metabolite      Reaction      Flux  C-Number   C-Flux
     ca2_e      EX_ca2_e  0.004565         0    0.00%
      cl_e       EX_cl_e  0.004565         0    0.00%
 cobalt2_e  EX_cobalt2_e 2.192E-05         0    0.00%
     cu2_e      EX_cu2_e 0.0006218         0    0.00%
     fe2_e      EX_fe2_e   0.01409         0    0.00%
  glc__D_e   EX_glc__D_e        10         6  100.00%
       k_e        EX_k_e    0.1712         0    0.00%
     mg2_e      EX_mg2_e  0.007608         0    0.00%
     mn2_e      EX_mn2_e  0.000606         0    0.00%
    mobd_e     EX_mobd_e 6.139E-06         0    0.00%
     nh4_e      EX_nh4_e     9.471         0    0.00%
     ni2_e      EX_ni2_e 0.0002833         0    0.00%
      o2_e       EX_o2_e     22.13         0    0.00%
      pi_e       EX_pi_e     0.846         0    0.00%
     so4_e      EX_so4_e    0.2208         0    0.00%
     zn2_e      EX_zn2_e 0.0002991         0    0.00%



In [5]:
cobra_model = model.copy()                              # cópia do modelo 

Alteração das condições ambientais: 

In [6]:
cobra_model.reactions.get_by_id("EX_o2_e").upper_bound = 1000
cobra_model.reactions.get_by_id("EX_o2_e").lower_bound = 0
cobra_model.reactions.get_by_id("EX_glc__D_e").upper_bound = 1000
cobra_model.reactions.get_by_id("EX_glc__D_e").lower_bound = -15

In [7]:
cobra_model.reactions.get_by_id('EX_o2_e')              # Confirmação da alteração das condições ambientais no modelo

Reaction identifier,EX_o2_e
Name,O2 exchange
Memory address,0x02965913e748
Stoichiometry,o2_e --> O2 O2 -->
GPR,
Lower bound,0
Upper bound,1000


In [9]:
print(cobra_model.summary())                            # Sumário do modelo após a alteração das condições ambientais

Objective
1.0 BIOMASS_Ec_iML1515_core_75p37M = 0.2660981251911781

Uptake
------
Metabolite      Reaction      Flux  C-Number  C-Flux
     ca2_e      EX_ca2_e  0.001385         0   0.00%
      cl_e       EX_cl_e  0.001385         0   0.00%
     co2_e      EX_co2_e   0.09771         1   0.11%
 cobalt2_e  EX_cobalt2_e 6.652E-06         0   0.00%
     cu2_e      EX_cu2_e 0.0001887         0   0.00%
     fe2_e      EX_fe2_e  0.002196         0   0.00%
     fe3_e      EX_fe3_e  0.002078         0   0.00%
  glc__D_e   EX_glc__D_e        15         6  99.89%
     h2o_e      EX_h2o_e     5.865         0   0.00%
       k_e        EX_k_e   0.05194         0   0.00%
     mg2_e      EX_mg2_e  0.002308         0   0.00%
     mn2_e      EX_mn2_e 0.0001839         0   0.00%
    mobd_e     EX_mobd_e 1.863E-06         0   0.00%
     nh4_e      EX_nh4_e     2.874         0   0.00%
     ni2_e      EX_ni2_e 8.595E-05         0   0.00%
      pi_e       EX_pi_e    0.2567         0   0.00%
     so4_e      EX

In [8]:
solution = cobra_model.optimize()  
solution.objective_value

0.2660981251911781

In [12]:
acetato_e =model.metabolites.get_by_id('ac_e')
print(acetato_e.summary())                              # Verificação das reações que envolvem o metabolito acetato extracelular

ac_e
====
Formula: C2H3O2

Producing Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []

Consuming Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []


In [13]:
from cobra.util.solver import linear_reaction_coefficients
linear_reaction_coefficients(cobra_model)

{<Reaction BIOMASS_Ec_iML1515_core_75p37M at 0x296590e5988>: 1.0}

Com as condições ambientais determinadas, os fluxos da reação que conduz o acetato para o meio extracelular e da biomassa são dados por:

In [14]:
solution.fluxes['EX_ac_e']                                  # Fluxo da reação EX_ac_e (acetato para o meio extracelular) com a otimização da biomassa

13.025232770730115

In [15]:
solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']           # Fluxo da biomassa com a otimização da biomassa

0.2660981251911781